<a href="https://colab.research.google.com/github/lkarjun/Data-Science-from-Scratch/blob/master/23%20Recommender%20Systems/recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Recommending system

In [15]:
users_interests = [["Hadoop", "Big Data", "HBase", "Java", "Spark","Storm", "Cassandra"],
                   ["NoSQL", "MongoDB", "Cassandra", "HBase","Postgres"],
                   ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
                   ["R", "Python", "statistics", "regression", "probability"],
                   ["machine learning", "regression", "decision trees", "libsvm"],
                   ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
                   ["statistics", "probability", "mathematics", "theory"],
                   ["machine learning", "scikit-learn", "Mahout", "neural networks"],
                   ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
                   ["Hadoop", "Java", "MapReduce", "Big Data"],
                   ["statistics", "R", "statsmodels"],
                   ["C++", "deep learning", "artificial intelligence", "probability"],
                   ["pandas", "R", "Python"],
                   ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
                   ["libsvm", "regression", "support vector machines"]]

### Recommending what's popular

In [3]:
from collections import Counter

In [4]:
popular_interests = Counter(interest
                            for user_interests in users_interests
                            for interest in user_interests)

In [5]:
from typing import List, Tuple

In [6]:
def most_popular_new_interests(
                user_interests: List[str],
                max_results: int = 5) -> List[Tuple[str, int]]:

                suggestions = [(interest, frequency)
                                for interest, frequency in popular_interests.most_common()
                                if interest not in user_interests]

                return suggestions[:max_results]


In [7]:
most_popular_new_interests(users_interests[1])

[('Python', 4), ('R', 4), ('Big Data', 3), ('Java', 3), ('statistics', 3)]

### User-Based Collabrative Filtering

In [8]:
unique_interests = sorted({interest for user_interests in users_interests
                           for interest in user_interests})

In [9]:
unique_interests[:4]

['Big Data', 'C++', 'Cassandra', 'HBase']

In [10]:
def make_user_interest_vector(user_interests: List[str]) -> List[int]:
  '''given a list of interests, produce a vector whose ith element is 1
     if unique_interests[i] is in the list, 0 otherwise'''
  return [1 if interest in user_interests else 0
             for interest in unique_interests]

In [17]:
user_interest_vectors = [make_user_interest_vector(user_interests)
                         for user_interests in users_interests]

In [20]:
from linearalgebra import dot
import math

def cosine_similarity(v1, v2) -> float:
    return dot(v1, v2) / math.sqrt(dot(v1, v1) * dot(v2, v2))


user_similarities = [[cosine_similarity(interest_vector_i,                                                      interest_vector_j)
                      for interest_vector_j in user_interest_vectors]
                      for interest_vector_i in user_interest_vectors]

In [21]:
def most_similar_users_to(user_id: int) -> List[Tuple[int, float]]:
    pairs = [(other_user_id, similarity)
            for other_user_id, similarity in enumerate(user_similarities[user_id])
            if user_id != other_user_id and similarity > 0]

    return sorted(pairs, key=lambda pair: pair[1], reverse=True)

In [22]:
most_similar_users_to(0)

[(9, 0.5669467095138409),
 (1, 0.3380617018914066),
 (8, 0.1889822365046136),
 (13, 0.1690308509457033),
 (5, 0.1543033499620919)]

In [23]:
from collections import defaultdict

In [26]:
def user_based_suggestions(user_id: int, include_current_interests: bool = False):
    suggestions: Dict[str, float] = defaultdict(float)
    
    for other_user_id, similarity in most_similar_users_to(user_id):
        for interest in users_interests[other_user_id]:
            suggestions[interest] += similarity
    
    suggestions = sorted(suggestions.items(), 
                         key=lambda pairs: pairs[-1], reverse=True)
    
    if include_current_interests:
        return suggestions
    else:
        return [(suggestions, weight)
                for suggestions, weight in suggestions
                if suggestions not in users_interests[user_id]]

In [27]:
user_based_suggestions(0)

[('MapReduce', 0.5669467095138409),
 ('MongoDB', 0.50709255283711),
 ('Postgres', 0.50709255283711),
 ('NoSQL', 0.3380617018914066),
 ('neural networks', 0.1889822365046136),
 ('deep learning', 0.1889822365046136),
 ('artificial intelligence', 0.1889822365046136),
 ('databases', 0.1690308509457033),
 ('MySQL', 0.1690308509457033),
 ('Python', 0.1543033499620919),
 ('R', 0.1543033499620919),
 ('C++', 0.1543033499620919),
 ('Haskell', 0.1543033499620919),
 ('programming languages', 0.1543033499620919)]

## Item - Based collaborative Filtering

In [28]:
interest_user_matrix = [[user_interest_vector[j]
                        for user_interest_vector in                   user_interest_vectors]
                        for j, _ in enumerate(unique_interests)]

In [31]:
interest_user_matrix[0]

[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]

In [33]:
interest_similarities = [[cosine_similarity(user_vector_i, user_vector_j)
             for user_vector_j in interest_user_matrix]
             for user_vector_i in interest_user_matrix]

In [34]:
def most_similar_interests_to(interest_id: int):

    similarities = interest_similarities[interest_id]

    pairs = [(unique_interests[other_interest_id], similarity)
             for other_interest_id, similarity in enumerate(similarities)
             if interest_id != other_interest_id and similarity > 0]

    return sorted(pairs, key=lambda pairs: pairs[-1], reverse=True)

In [35]:
most_similar_interests_to(0)

[('Hadoop', 0.8164965809277261),
 ('Java', 0.6666666666666666),
 ('MapReduce', 0.5773502691896258),
 ('Spark', 0.5773502691896258),
 ('Storm', 0.5773502691896258),
 ('Cassandra', 0.4082482904638631),
 ('artificial intelligence', 0.4082482904638631),
 ('deep learning', 0.4082482904638631),
 ('neural networks', 0.4082482904638631),
 ('HBase', 0.3333333333333333)]

In [37]:
def item_based_suggestions(user_id: int, include_current_interests: bool = False):

    suggestions = defaultdict(float)

    user_interest_vector = user_interest_vectors[user_id]

    for interest_id, is_interested in enumerate(user_interest_vector):
        if is_interested == 1:
            similar_interests = most_similar_interests_to(interest_id)
            for interest, similarity in similar_interests:
                suggestions[interest] += similarity
    
    suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[-1], reverse=True)
    
    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestions not in users_interests[user_id]]

In [38]:
item_based_suggestions(0)

[('Spark', 4.146264369941973),
 ('Storm', 4.146264369941973),
 ('Hadoop', 3.9554550146924106),
 ('Cassandra', 3.547206724228547),
 ('Java', 3.3794454097708404),
 ('Big Data', 3.3794454097708404),
 ('HBase', 3.0461120764375074),
 ('MapReduce', 1.861807319565799),
 ('MongoDB', 1.3164965809277263),
 ('Postgres', 1.3164965809277263),
 ('NoSQL', 1.2844570503761732),
 ('MySQL', 0.5773502691896258),
 ('databases', 0.5773502691896258),
 ('Haskell', 0.5773502691896258),
 ('programming languages', 0.5773502691896258),
 ('artificial intelligence', 0.4082482904638631),
 ('deep learning', 0.4082482904638631),
 ('neural networks', 0.4082482904638631),
 ('C++', 0.4082482904638631),
 ('Python', 0.2886751345948129),
 ('R', 0.2886751345948129)]

In [40]:

def main():
    
    # Replace this with the locations of your files
    
    # This points to the current directory, modify if your files are elsewhere.
    MOVIES = "u.item"   # pipe-delimited: movie_id|title|...
    RATINGS = "u.data"  # tab-delimited: user_id, movie_id, rating, timestamp
    
    from typing import NamedTuple
    
    class Rating(NamedTuple):
        user_id: str
        movie_id: str
        rating: float
    
    import csv
    # We specify this encoding to avoid a UnicodeDecodeError.
    # see: https://stackoverflow.com/a/53136168/1076346
    with open(MOVIES, encoding="iso-8859-1") as f:
        reader = csv.reader(f, delimiter="|")
        movies = {movie_id: title for movie_id, title, *_ in reader}
    
    # Create a list of [Rating]
    with open(RATINGS, encoding="iso-8859-1") as f:
        reader = csv.reader(f, delimiter="\t")
        ratings = [Rating(user_id, movie_id, float(rating))
                   for user_id, movie_id, rating, _ in reader]
    
    # 1682 movies rated by 943 users
    assert len(movies) == 1682
    assert len(list({rating.user_id for rating in ratings})) == 943
    
    import re
    
    # Data structure for accumulating ratings by movie_id
    star_wars_ratings = {movie_id: []
                         for movie_id, title in movies.items()
                         if re.search("Star Wars|Empire Strikes|Jedi", title)}
    
    # Iterate over ratings, accumulating the Star Wars ones
    for rating in ratings:
        if rating.movie_id in star_wars_ratings:
            star_wars_ratings[rating.movie_id].append(rating.rating)
    
    # Compute the average rating for each movie
    avg_ratings = [(sum(title_ratings) / len(title_ratings), movie_id)
                   for movie_id, title_ratings in star_wars_ratings.items()]
    
    # And then print them in order
    for avg_rating, movie_id in sorted(avg_ratings, reverse=True):
        print(f"{avg_rating:.2f} {movies[movie_id]}")
    
    import random
    random.seed(0)
    random.shuffle(ratings)
    
    split1 = int(len(ratings) * 0.7)
    split2 = int(len(ratings) * 0.85)
    
    train = ratings[:split1]              # 70% of the data
    validation = ratings[split1:split2]   # 15% of the data
    test = ratings[split2:]               # 15% of the data
    
    avg_rating = sum(rating.rating for rating in train) / len(train)
    baseline_error = sum((rating.rating - avg_rating) ** 2
                         for rating in test) / len(test)
    
    # This is what we hope to do better than
    assert 1.26 < baseline_error < 1.27
    
    
    # Embedding vectors for matrix factorization model
    
    from scratch.deep_learning import random_tensor
    
    EMBEDDING_DIM = 2
    
    # Find unique ids
    user_ids = {rating.user_id for rating in ratings}
    movie_ids = {rating.movie_id for rating in ratings}
    
    # Then create a random vector per id
    user_vectors = {user_id: random_tensor(EMBEDDING_DIM)
                    for user_id in user_ids}
    movie_vectors = {movie_id: random_tensor(EMBEDDING_DIM)
                     for movie_id in movie_ids}
    
    
    # Training loop for matrix factorization model
    
    from typing import List
    import tqdm
    from scratch.linear_algebra import dot
    
    def loop(dataset: List[Rating],
             learning_rate: float = None) -> None:
        with tqdm.tqdm(dataset) as t:
            loss = 0.0
            for i, rating in enumerate(t):
                movie_vector = movie_vectors[rating.movie_id]
                user_vector = user_vectors[rating.user_id]
                predicted = dot(user_vector, movie_vector)
                error = predicted - rating.rating
                loss += error ** 2
    
                if learning_rate is not None:
                    #     predicted = m_0 * u_0 + ... + m_k * u_k
                    # So each u_j enters output with coefficent m_j
                    # and each m_j enters output with coefficient u_j
                    user_gradient = [error * m_j for m_j in movie_vector]
                    movie_gradient = [error * u_j for u_j in user_vector]
    
                    # Take gradient steps
                    for j in range(EMBEDDING_DIM):
                        user_vector[j] -= learning_rate * user_gradient[j]
                        movie_vector[j] -= learning_rate * movie_gradient[j]
    
                t.set_description(f"avg loss: {loss / (i + 1)}")
    
    learning_rate = 0.05
    for epoch in range(20):
        learning_rate *= 0.9
        print(epoch, learning_rate)
        loop(train, learning_rate=learning_rate)
        loop(validation)
    loop(test)
    
    
    from scratch.working_with_data import pca, transform
    
    original_vectors = [vector for vector in movie_vectors.values()]
    components = pca(original_vectors, 2)
    
    ratings_by_movie = defaultdict(list)
    for rating in ratings:
        ratings_by_movie[rating.movie_id].append(rating.rating)
    
    vectors = [
        (movie_id,
         sum(ratings_by_movie[movie_id]) / len(ratings_by_movie[movie_id]),
         movies[movie_id],
         vector)
        for movie_id, vector in zip(movie_vectors.keys(),
                                    transform(original_vectors, components))
    ]